<a href="https://colab.research.google.com/github/armarmarm927/dl_lecture_competition_pub/blob/main/sparkproto21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomAffine([-10,10],scale=(1,1.25)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomErasing(p=0.5,scale=(0.02,0.33),ratio=(0.3,3.3),value=0,inplace=False)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [16]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [17]:
model_ft = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
model_ft.fc = nn.Linear(model_ft.fc.in_features, 10)
net = model_ft.to(device)

In [18]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.Adam(net.parameters(), lr=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [19]:
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [ ]:
num_epochs = 70
loss_list = []
acc_list = []
train_acc_list = []

for epoch in range(num_epochs):
    net.train()
    correct_train = 0
    total_train = 0
    epoch_loss = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        inputs, targets_a, targets_b, lam = mixup_data(inputs, labels, alpha=1.0)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    train_accuracy = 100. * correct_train / total_train
    train_acc_list.append(train_accuracy)
    print(f'Epoch {epoch+1}: Loss: {epoch_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%')
    loss_list.append(epoch_loss/len(train_loader))

    net.eval()
    correct = 0
    total = 0
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    test_accuracy = 100. * correct / total
    acc_list.append(test_accuracy)
    print(f'Test Accuracy: {test_accuracy:.2f}%')

    # 混同行列の出力
    cm = confusion_matrix(y_true, y_pred)
    print(f'Confusion Matrix (Epoch {epoch+1}):')
    print(cm)

    scheduler.step()
torch.save(net.state_dict(), '/content/drive/MyDrive/Weight_Epoch50.pth')

Epoch 1: Loss: 1.8535, Train Accuracy: 33.30%
Test Accuracy: 73.78%
Confusion Matrix (Epoch 1):
[[857  17  21  11   5   5   3   5  69   7]
 [ 20 897   3   8  18  14   0   6  16  18]
 [ 67   1 678  34  77  86  40  13   4   0]
 [ 26   3  68 477  45 316  33  12  17   3]
 [ 33   2  85  45 661  54  50  54  15   1]
 [ 14   0  37  77  39 804   4  20   4   1]
 [  8   5  52  46  48  49 776   6  10   0]
 [ 15   6  39  28  54 113   1 729   5  10]
 [ 75  31  13   7   7   7   0   2 850   8]
 [ 64 187   4  18   1  15   2  12  48 649]]


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 2: Loss: 1.6727, Train Accuracy: 39.20%
Test Accuracy: 76.44%
Confusion Matrix (Epoch 2):
[[849  26  33   3   2   0   3  11  53  20]
 [  8 907   7   3   0   0   2   2  14  57]
 [ 58   7 738  10  28  28  90  30   3   8]
 [ 28  11 109 415  22 251  72  67  12  13]
 [ 37   3 158  22 592  23  87  68   8   2]
 [ 14   6  96  40  22 699  26  89   4   4]
 [  9   8  85  21   2  11 854   7   3   0]
 [ 19   6  43   3  29  21   8 859   1  11]
 [ 58  22   8   5   1   0   4   3 879  20]
 [ 18  82   6   6   2   1   3   5  25 852]]


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 3: Loss: 1.6054, Train Accuracy: 40.45%
Test Accuracy: 79.91%
Confusion Matrix (Epoch 3):
[[902  16  15   5   1   0   3   7  32  19]
 [  5 921  10   3   0   1   1   0  31  28]
 [ 74   7 690  36  33  43  82  21   8   6]
 [ 20  15  67 648  21 129  37  34  18  11]
 [ 27   6  72  56 659  29  66  78   7   0]
 [ 14   3  43 136  23 715  12  48   2   4]
 [ 11   4  47  42   3  12 867   6   7   1]
 [ 19   7  30  26  18  26   0 862   2  10]
 [ 77  17   6   5   1   1   1   2 883   7]
 [ 30  93   2   6   0   1   2   2  20 844]]


...

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 41: Loss: 1.3792, Train Accuracy: 48.60%
Test Accuracy: 89.40%
Confusion Matrix (Epoch 41):
[[949   3  10   3   3   0   0   4  20   8]
 [  8 967   0   1   0   1   1   1   4  17]
 [ 26   0 917  15  13  13  11   4   1   0]
 [ 16   3  39 758  21 116  21  18   6   2]
 [ 11   2  32  24 861  15  22  29   3   1]
 [  7   1  18  79  17 842  16  19   0   1]
 [  6   3  29  18   5   9 927   2   1   0]
 [  7   0   9   9  10  17   2 879  66   1]
 [ 41  10   3   4   0   0   0   1 936   5]
 [ 23  44   3   8   0   3   0   1  14 904]]


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 42: Loss: 1.3969, Train Accuracy: 48.43%
Test Accuracy: 88.71%
Confusion Matrix (Epoch 42):
[[930   4  20   5   4   0   3   2  21  11]
 [  6 970   1   1   0   1   0   1   3  17]
 [ 20   0 925  15  13  12  11   3   1   0]
 [ 10   3  39 788  21 105  18  11   3   2]
 [  5   2  41  26 866  17  18  22   3   0]
 [  4   1  18  86  18 846  11  15   0   1]
 [  5   3  33  17   6  11 922   2   1   0]
 [  4   0  13  13  14  26   3 780 146   1]
 [ 32  10   5   6   0   0   0   1 939   7]
 [ 22  45   3   7   0   3   0   1  14 905]]


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
plt.plot(acc_list)
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.xticks(np.arange(0,200,20))
plt.ylabel('Accuracy')
plt.show()

In [ ]:
plt.plot(train_acc_list)
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.xticks(np.arange(0,200,20))
plt.ylabel('Accuracy')
plt.show()

In [ ]:
plt.plot(loss_list)
plt.title('Cross Entropy Loss')
plt.xlabel('Epoch')
plt.xticks(np.arange(0,200,20))
plt.ylabel('Accuracy')
plt.show()